# Tropical hypersurfaces in polymake

This notebook illustrates how the coordinates of tropical hypersurfaces in polymake work. Furthermore it takes a look at the implications of looking at the hypersurface in the projective torus.

The question resulting in this notebook was about a point on a tropical hypersurface that did not seem to be contained in it.

In [1]:
application "tropicaal";

First I reproduce the example in smaller dimension.

In [2]:
#Tropical condition corresponding to {0 1 2 3 4 5}
$h31 = new Vector<Int>(6);
$h31->[0]=1;
$h31->[5]=1;

$h32 = new Vector<Int>(6);
$h32->[2]=1;
$h32->[3]=1;

$h33 = new Vector<Int>(6);
$h33->[1]=1;
$h33->[4]=1;

$H3 = new Hypersurface<Max>(MONOMIALS=> 
[$h31,$h32,$h33], COEFFICIENTS=>[0,0,0]);

Now let us print the properties:

In [18]:
print $H3->RAYS;

1 0 0 0 0 0 0
0 0 1 1 1 1 1
0 0 -1 0 0 0 0
0 0 0 -1 0 0 0


In [19]:
print $H3->LINEALITY_SPACE;

0 0 0 -1 1 0 0
0 0 -1 0 0 1 0
0 -1 0 0 0 0 1


In [20]:
print $H3->MAXIMAL_POLYTOPES;

{0 2}
{0 3}
{0 1}


In [21]:
print $H3->POLYNOMIAL;

x_0*x_5 + x_1*x_4 + x_2*x_3

The first thing to notice is that this does not really fit with the rays. So there is an additional shift incorporated, meaning that the variable `x_i` actually corresponds to the $(i+1)$-th coordinate of the ambient space.

E.g. the ray `0 0 -1 0 0 0 0` means: Set `x_1 = -1`. Then the second term of the polynomial is lowest and the remaining two terms are zero and maximal, so this is indeed a point on our hypersurface.

But now we get to the main question: With this data I can set every coordinate to be $-1$ except for `x_0`. Instead we have this awkward vector `0 0 1 1 1 1 1` meaning setting everything but `x_0` to be 1. It serves the same purpose, but as you can see below the equivalent vector `0 -1 0 0 0 0 0` is not contained.

In [13]:
# $v = new Vector(qw(0   0 0 0 1 0 1));
$v = new Vector(qw(0   -1 0 0 0 0 0));
print polynomial_vanishes($H3->POLYNOMIAL, new Vector<TropicalNumber<Max>>($v->slice(range(1,6)))),"\n";

true


First check whether it is a tropical zero.

In [14]:
for(my $i=0; $i<3; $i++){
    my $P = $H3->polytope($i);
    print $P->contains($v),"\n";
}

false
false
false


But it is not contained in the polyhedral structure.

The reason for this can be found in the lineality space. It should contain the vector `0 1 1 1 1 1 1` where we set everything to be `1`, but it does not. Now read the manual:

https://polymake.org/doku.php/documentation/master/tropical/hypersurface

It explicitly says that this solution gets modded out, since it is trivial. Putting it back in makes the fan_containment procedure work

In [22]:
$R = $H3->RAYS;
$L = new Matrix($H3->LINEALITY_SPACE / (zero_vector(1) | ones_vector(6)));
$MP = $H3->MAXIMAL_POLYTOPES;
$H4 = new fan::PolyhedralComplex(INPUT_RAYS=>$R, INPUT_LINEALITY=>$L, INPUT_POLYTOPES=>$MP);

In [23]:
for(my $i=0; $i<3; $i++){
    my $P = $H4->polytope($i);
    print $P->contains($v),"\n";
}

false
false
true


With this approach also your vector is contained:

In [24]:
$v = new Vector(qw(0 0 0 0 1 0 1));
for(my $i=0; $i<3; $i++){
    my $P = $H4->polytope($i);
    print $P->contains($v),"\n";
}

true
false
false
